In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
# Seed value
seed_value = 55

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value)

session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

In [2]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [3]:
dataset_file_name = 'human_dataset.txt'
y_train_full = np.load('../Saved Data/Training Set/labels.npy')
X_train_full = np.load('../Saved Data/Training Set/dna_spectral_representation.npy')

In [4]:
k = 5
input_vector_size = np.power(4, k)

In [5]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=0.1,
                                                        random_state=20)

# Model : LeNet5 like Architecture CNN for Text Classification

In [6]:
model = keras.Sequential()

model.add(keras.layers.Conv1D(filters=30, kernel_size=(5), activation='relu', input_shape=(input_vector_size, 1),padding="same"))
model.add(keras.layers.MaxPooling1D())
model.add(keras.layers.Dropout(0.3))

model.add(keras.layers.Conv1D(filters=15, kernel_size=(5), activation='relu', input_shape=(int(input_vector_size / 2), 1), padding="same"))
model.add(keras.layers.MaxPooling1D())
model.add(keras.layers.Dropout(0.3))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(units=256, activation='relu'))
model.add(keras.layers.Dropout(0.3))

model.add(keras.layers.Dense(units=128, activation='relu'))
model.add(keras.layers.Dropout(0.3))

model.add(keras.layers.Dense(units=64, activation='relu'))

model.add(keras.layers.Dense(units=int(7), activation = 'softmax'))

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 1024, 30)          180       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 512, 30)          0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 512, 30)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 512, 15)           2265      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 256, 15)          0         
 1D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 256, 15)           0

In [8]:
opt = keras.optimizers.SGD(learning_rate=0.045, momentum=0.0)
model.compile(loss="sparse_categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [9]:
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid), batch_size=32)

Epoch 1/30
99/99 [==============================] - 6s 55ms/step - loss: 1.8667 - accuracy: 0.2848 - val_loss: 1.7618 - val_accuracy: 0.3105
Epoch 2/30
99/99 [==============================] - 4s 42ms/step - loss: 1.6993 - accuracy: 0.3536 - val_loss: 1.5213 - val_accuracy: 0.4302
Epoch 3/30
99/99 [==============================] - 4s 39ms/step - loss: 1.5455 - accuracy: 0.4393 - val_loss: 1.3747 - val_accuracy: 0.5499
Epoch 4/30
99/99 [==============================] - 4s 40ms/step - loss: 1.3884 - accuracy: 0.4967 - val_loss: 1.1323 - val_accuracy: 0.6268
Epoch 5/30
99/99 [==============================] - 4s 41ms/step - loss: 1.2536 - accuracy: 0.5550 - val_loss: 1.0570 - val_accuracy: 0.5954
Epoch 6/30
99/99 [==============================] - 4s 43ms/step - loss: 1.1460 - accuracy: 0.6032 - val_loss: 0.8814 - val_accuracy: 0.6781
Epoch 7/30
99/99 [==============================] - 4s 41ms/step - loss: 1.0681 - accuracy: 0.6302 - val_loss: 0.7799 - val_accuracy: 0.7493
Epoch 8/30
99

In [10]:
# data = history.history
# pd.DataFrame(data).plot(figsize=(8, 6))
# plt.grid(True)
# plt.gca().set_ylim(0, 2)
# plt.show()

In [11]:
model.save("../Saved Data/Model/")
model.save("../Saved Data/Model Architecture/model.h5")

INFO:tensorflow:Assets written to: ../Saved Data/Model/assets


INFO:tensorflow:Assets written to: ../Saved Data/Model/assets
